In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
# from statsmodels.tsa.stattools import adfuller, kpss
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
import warnings
from fbprophet import Prophet

%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [2]:
'''
ROI
Mean day to pending
For sale investory
'''

'\nROI\nMean day to pending\nFor sale investory\n'

In [3]:
from fbprophet import Prophet

# ! pip install fbprophet

def fbprophet_func(df, periods=13):
    '''
    The input dataframe must only have two columns called ds and y

    Returns the Original and Predictions along with the RMSE of the test data

    Will split the data into a train, test for you.

    Height is for the height of the arrow

    periods is for the periods you wish to predict
    '''
    df_prophet = df
#     cutoff = round(df_prophet.shape[0] * train_size)
#     train = df_prophet[:cutoff]
#     test = df_prophet[cutoff:]

    model = Prophet()
    model.fit(df_prophet)
    forecast = model.predict(df_prophet)

    future = model.make_future_dataframe(periods=periods, freq='MS')
    future_forecast = model.predict(future)

    fig, ax = plt.subplots()

    sns.lineplot(df['ds'], df['y'],
                 label='Original', color='r', linewidth=4)
    sns.lineplot(
        future_forecast['ds'], future_forecast['yhat'], label='Predictions', color='b')
    
    model.plot(future_forecast, ax=ax)
    
    plt.vlines(x=df['ds'].max(), ymin=future_forecast['yhat'].min(
    ) - 5, ymax=future_forecast['yhat'].max() + 5, linestyles='dashed')
    
    ax.set_title('Denver House Prices')
    ax.set_ylabel('House Prices')
    ax.set_xlabel('Time')
    plt.legend(loc='best')

    MSE = np.square(np.subtract(df.y, future_forecast.yhat)).mean()

    RMSE = math.sqrt(MSE)
    print(f'RMSE for the train data: {RMSE}')
    print("%RMSE: ", RMSE / df.mean())

In [4]:
def LSTM_func(df, City=None, verbose=1, use_multiprocessing=False, epochs=5, batch_size=5, test_size=0.8):
    '''
    The dataframe can only have two columns, time and values drop everthing else before using this function
    you will break it!!

    Parameters
    -----------
    df = Dataframe (DON'T FORGET ONLY TIME AND VALUES)
    City = The name of the city you are modeling

    Returns
    ---------
    A plot
    '''
    cols = list(df)[1:]

    train, test = df[:int(
        (len(df) * .80))], df[int(round(len(df) * test_size)):]

    train_float = train[cols].astype(float)
    test_float = test[cols].astype(float)
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_float)
    test_scaled = scaler.transform(test_float)

    sequence_length = 5
    window_length = sequence_length + 1

    x_train = []
    y_train = []
    for i in range(0, len(train) - window_length + 1):
        window = train_scaled[i:i + window_length]
        x_train.append(window[:-1, :])
        y_train.append(window[-1, [-1]])
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_test = []
    y_test = []
    for i in range(0, len(test) - window_length + 1):
        window = test_scaled[i:i + window_length, :]
        x_test.append(window[:-1, :])
        y_test.append(window[-1, [-1]])
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    # Model building starts here
    model = Sequential()
    model.add(LSTM(64, activation='relu',
                   input_shape=(5, 1), return_sequences=True))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(LSTM(16, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    model.summary()
    # Fitting the model on the X_train and y_train
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(
        x_test, y_test), verbose=verbose, use_multiprocessing=use_multiprocessing)

    y_predicts = history.predict(x_train)
    y_predict_inv_scaled = scaler.inverse_transform(y_predicts)
    y_predict = []
    for predict in y_predict_inv_scaled:
        y_predict.append(predict[0])

    y_predict = pd.Series(y_predict, index=train.index[5:])

    y_test_inv_scaled = scaler.inverse_transform(y_train)
    y_train = []
    for predict in y_test_inv_scaled:
        y_train.append(predict[0])

    y_train = pd.Series(y_train, index=train.index[5:])
    
    MSE = np.square(np.subtract(y_train,y_predict)).mean() 
 
    RMSE = math.sqrt(MSE)
    print(f'RMSE: {RMSE}')
    print("%RMSE: ", RMSE/y_test.mean())
    return history 

In [5]:
def train_test_split(ts, train_size=.8):
    """
    Takes in a time series and creates and train test split.
    
    Parameters
    ----------
    
    ts: time series
    train size: (float) portion of time series reserved for train (default=.8)
    """
    cutoff = round(ts.shape[0]*.8)
    train = ts[:cutoff]
    test = ts[cutoff:]
    
#     return train, test